In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('src/data_tms.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
df = df.dropna(subset=['Age', 'Embarked'])
df1 = df.drop(columns=['PassengerId', 'Survived', 'Name', 'Ticket', 'Fare', 'Cabin'])

In [63]:
df1.Sex = df.Sex.map({
    'male': 1,
    'female': 0
})
df1.Embarked = df.Embarked.map({
    'S': 0,
    'C': 1,
    'Q': 2
})

In [51]:
X = df1.to_numpy()
y = df.Survived.to_numpy()

In [50]:
model = LogisticRegression()
model.fit(X, y)

model_y_sk = model.predict(X)

model.score(X, y)

0.8089887640449438

In [ ]:
def sigmoid(z):
    # Ограничиваем z, чтобы избежать переполнения
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def logistic_regression_np(
        X: np.ndarray,
        y: np.ndarray
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Обучает логистическую регрессию с использованием градиентного спуска.

    :param X: Матрица признаков формы (m, n), где m — количество примеров,
        n — количество признаков.
    :param y: Вектор целевых значений формы (m,), содержащий 0 или 1.
    :return: Кортеж из трёх элементов:
        - weights: Вектор весов модели (включая свободный член) формы (n + 1,).
        - X_mean: Вектор средних значений по каждому признаку,
            использованный для нормализации.
        - X_std: Вектор стандартных отклонений по каждому признаку,
            использованный для нормализации.
    """
    # Нормализация признаков
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std[X_std == 0] = 1
    X_normalized = (X - X_mean) / X_std
    
    # Добавление столбца единиц для свободного члена
    m, n = X_normalized.shape
    X_b = np.column_stack([np.ones(m), X_normalized])

    learning_rate = 0.001
    weights = np.random.randn(n + 1) * 0.01

    for i in range(50000):
        z = X_b @ weights
        y_pred = sigmoid(z)
        error = y_pred - y

        gradient = (1 / m) * X_b.T @ error
        weights = weights - learning_rate * gradient

    return weights, X_mean, X_std


def predict_logistic_np(X, weights, X_mean, X_std, threshold=0.5):
    """
    Выполняет предсказание на новых данных с использованием обученной модели
    логистической регрессии.

    :param X: Матрица признаков для предсказания формы (m, n).
    :param weights: Вектор весов модели (включая свободный член) формы (n + 1,).
    :param X_mean: Вектор средних значений признаков, полученный при обучении.
    :param X_std: Вектор стандартных отклонений признаков, полученный при обучении.
    :param threshold: Порог для бинарной классификации (по умолчанию 0.5).
    :return: Вектор предсказанных вероятностей или меток формы (m,).
    """
    X_normalized = (X - X_mean) / X_std
    m = X_normalized.shape[0]
    X_b = np.column_stack([np.ones(m), X_normalized])
    
    z = X_b @ weights
    probabilities = sigmoid(z)

    return (probabilities >= threshold).astype(int)

In [65]:
weights, X_mean, X_std = logistic_regression_np(X, y)

pred_y = predict_logistic_np(X, weights, X_mean, X_std)

Точность предсказаний

In [54]:
(y == pred_y).mean()

np.float64(0.8019662921348315)

In [62]:
df1['Survived'] = df.Survived
df1['pred_y_sk'] = model_y_sk
df1['pred_y'] = pred_y

df1.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked,Survived,pred_y_sk,pred_y
0,3,1,22.0,1,0,0,0,0,0
1,1,0,38.0,1,0,1,1,1,1
2,3,0,26.0,0,0,0,1,1,1
3,1,0,35.0,1,0,0,1,1,1
4,3,1,35.0,0,0,0,0,0,0


Количество значений, которые не соответствуют действительному 'y' модели sklearn

In [60]:
(df1.pred_y_sk != df1.Survived).sum()

np.int64(136)

Количество значений, который не соответствуют действительному 'y' у моей модели

In [61]:
(df1.pred_y != df1.Survived).sum()

np.int64(141)